In [1]:
from theano import gpuarray as cuda

In [23]:
%matplotlib inline
from importlib import reload  # Python 3
import utils; reload(utils)
from utils import *
from keras.layers.merge import dot, add, concatenate

np.random.seed(42)


TypeError: 'int' object is not callable

In [3]:
# path = "data/ml-latest-small/"  
path = 'data/aclImdb_v1/aclImdb/'
model_path = path + 'models/'
if not os.path.exists(model_path): 
    os.mkdir(model_path)
batch_size=64

In [4]:
from keras.datasets import imdb
idx = imdb.get_word_index()

In [5]:
idx_arr = sorted(idx, key=idx.get)
idx_arr[:10]

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [6]:
idx2word = {v: k for k, v in idx.items()}

In [7]:
path = get_file('imdb_full.pkl',
                origin='https://s3.amazonaws.com/text-datasets/imdb_full.pkl',
                md5_hash='d091312047c43cf9e4e38fef92437263')
f = open(path, 'rb')
(x_train, labels_train), (x_test, labels_test) = pickle.load(f)

In [8]:
len(x_train)

25000

In [9]:
', '.join(map(str, x_train[0]))

'23022, 309, 6, 3, 1069, 209, 9, 2175, 30, 1, 169, 55, 14, 46, 82, 5869, 41, 393, 110, 138, 14, 5359, 58, 4477, 150, 8, 1, 5032, 5948, 482, 69, 5, 261, 12, 23022, 73935, 2003, 6, 73, 2436, 5, 632, 71, 6, 5359, 1, 25279, 5, 2004, 10471, 1, 5941, 1534, 34, 67, 64, 205, 140, 65, 1232, 63526, 21145, 1, 49265, 4, 1, 223, 901, 29, 3024, 69, 4, 1, 5863, 10, 694, 2, 65, 1534, 51, 10, 216, 1, 387, 8, 60, 3, 1472, 3724, 802, 5, 3521, 177, 1, 393, 10, 1238, 14030, 30, 309, 3, 353, 344, 2989, 143, 130, 5, 7804, 28, 4, 126, 5359, 1472, 2375, 5, 23022, 309, 10, 532, 12, 108, 1470, 4, 58, 556, 101, 12, 23022, 309, 6, 227, 4187, 48, 3, 2237, 12, 9, 215'

In [10]:
idx2word[23022]

'bromwell'

In [11]:
labels_train[-10:]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [12]:
vocab_size = 5000

trn = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_train]
test = [np.array([i if i<vocab_size-1 else vocab_size-1 for i in s]) for s in x_test]

In [13]:
lens = np.array([len(t) for t in trn])
(lens.max(), lens.min(), lens.mean())

(2493, 10, 237.71364)

In [14]:
seq_len = 500

trn = sequence.pad_sequences(trn, maxlen=seq_len, value=0)
test = sequence.pad_sequences(test, maxlen=seq_len, value=0)

In [15]:
trn.shape

(25000, 500)

In [16]:
model = Sequential([
    Embedding(vocab_size, 32, input_length=seq_len),
    Flatten(),
    Dense(100, activation='relu'),
    Dropout(0.7),
    Dense(1, activation='sigmoid')])

In [17]:
model.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               1600100   
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 1,760,201
Trainable params: 1,760,201
Non-trainable params: 0
_________________________________________________________________


In [18]:
trn

array([[   0,    0,    0, ...,   12,    9,  215],
       [   0,    0,    0, ...,    5,  336,  406],
       [   0,    0,    0, ...,    6,  176,  397],
       ...,
       [   0,    0,    0, ...,   17,   96,   75],
       [   0,    0,    0, ..., 4999, 1195,  795],
       [   0,    0,    0, ...,   11,    6, 1347]], dtype=int32)

In [19]:
model.fit(trn, labels_train, validation_data=(test, labels_test), epochs=2, batch_size=64)

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
25000/25000 [==============================] - 29s 1ms/step - loss: 0.4803 - acc: 0.7370 - val_loss: 0.2996 - val_acc: 0.8712
Epoch 2/2
25000/25000 [==============================] - 28s 1ms/step - loss: 0.2034 - acc: 0.9236 - val_loss: 0.2987 - val_acc: 0.8756
